

    
<div class="alert alert-block alert-info">
    <b>Author: Atef Bader</b> <br> 
    <b>Last Edit: 7/10/2020</b>
</div>
        

<div class="alert alert-block alert-danger"><b>Warning:</b> If you want to rerun the script, make sure to drop the <b>trans4cust</b> table from the command prompt of sqlite3 shell before you rerun the script.</div>

# Deliverables:

- Submit a single zip-compressed file that has the name: YourLastName_Exercise_3 that has the following files:

 1. Your **PDF document** that has your Source code and output
 2. Your **ipynb script** that has your Source code and output


# Objectives:

In this exercise, you will:

 - Analyze the dataset in the given CSV file
 - Clean the given dataset
 - Load the dataset into sqlite database engine
 - Execute different SQL queries




# Submission Formats :

Create a folder or directory with all supplementary files with your last name at the beginning of the folder name, compress that folder with zip compression, and post the zip-archived folder under the assignment link in Canvas. The following files should be included in an archive folder/directory that is uploaded as a single zip-compressed file. (Use zip, not StuffIt or any 7z or any other compression method.)


1. Complete IPYNB script that has the source code in Python used to access and analyze the data. The code should be submitted as an IPYNB script that can be be loaded and run in Jupyter Notebook for Python
2. Output from the program, such as console listing/logs, text files, and graphics output for visualizations. If you use the Data Science Computing Cluster or School of Professional Studies database servers or systems, include Linux logs of your sessions as plain text files. Linux logs may be generated by using the script process at the beginning of your session, as demonstrated in tutorial handouts for the DSCC servers.
3. List file names and descriptions of files in the zip-compressed folder/directory.


Formatting Python Code
When programming in Python, refer to Kenneth Reitz’ PEP 8: The Style Guide for Python Code:
http://pep8.org/ (Links to an external site.)Links to an external site.
There is the Google style guide for Python at
https://google.github.io/styleguide/pyguide.html (Links to an external site.)Links to an external site.
Comment often and in detail.


###   Data Preparation

As a data scientist for BestDeal retailer, you
have been tasked with improving their revenue and the effectiveness of the
marketing campaign of their electronic products. The given dataset has
10,000 records for the purchases of their customers and is used to predict
customers shopping patterns and to provide answers for ad-hoc queries.
The dataset DirtyData4BestDeal10000.csv is drawn from its database of
customers.

In [1]:
import pandas as pd  # panda's nickname is pd

import numpy as np  # numpy as np

from pandas import DataFrame, Series     # for convenience

import sqlalchemy

from sqlalchemy import create_engine

from sqlalchemy import inspect

### Lets read the dirtydata4bestdeal CSV and load into a dataframe object

In [2]:
dirtydata4bestdeal=pd.read_csv('DirtyData4BestDeal10000.csv')

In [3]:

# Do you see NaN values below?

dirtydata4bestdeal.head(4)

,ZipCode,CustomerAge,SamsungTV46LED,SonyTV42LED,XBOX360,DellLaptop,BoseSoundSystem,BoseHeadSet,SonyHeadSet,iPod,...,GalaxyTablet,SurfaceTablet,HPLaptop,HDMICable,SpeakerCable,CallOfDutyGame,GrandTheftAutoGame,ASUSLaptop,LenevoLaptop,TVStandWallMount
0,30134.0,35.0,1,1,1,0,0,1.0,1.0,0.0,...,1,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1
1,62791.0,43.0,0,1,0,0,1,0.0,1.0,0.0,...,1,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1
2,60611.0,23.0,1,NaN,0,1,0,NaN,1.0,1.0,...,0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,1.0,1
3,60616.0,56.0,0,1,1,1,0,0.0,1.0,1.0,...,0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0


### Lets use boxplot to visualize the data and get an idea if there are dirty/messy/invalid data

In [21]:
dirtydata4bestdeal.boxplot(column='CustomerAge')


<AxesSubplot:>

In [22]:
dirtydata4bestdeal.boxplot(column='LenevoLaptop')

<AxesSubplot:>

In [23]:
dirtydata4bestdeal.boxplot(column='ZipCode')


<AxesSubplot:>

### Lets clean the dirty/messy data in the dirtydata4bestdeal dataframe object

In [22]:
# Drop the NaN values 

cleandata4bestdeal=dirtydata4bestdeal.dropna()
cleandata4bestdeal.head(10)

df = cleandata4bestdeal.copy()

# Do you see NaN values dropped below?



In [23]:
# GET ALL of these binary columns to be the same datatype
#df = df[df['DellLaptop'] != ' ']
df.head(10)

,ZipCode,CustomerAge,SamsungTV46LED,SonyTV42LED,XBOX360,DellLaptop,BoseSoundSystem,BoseHeadSet,SonyHeadSet,iPod,...,GalaxyTablet,SurfaceTablet,HPLaptop,HDMICable,SpeakerCable,CallOfDutyGame,GrandTheftAutoGame,ASUSLaptop,LenevoLaptop,TVStandWallMount
0,30134.0,35.0,1,1,1,0,0,1.0,1.0,0.0,...,1,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1
1,62791.0,43.0,0,1,0,0,1,0.0,1.0,0.0,...,1,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1
3,60616.0,56.0,0,1,1,1,0,0.0,1.0,1.0,...,0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0
5,2108.0,55.0,1,1,1,1,10,0.0,0.0,0.0,...,1,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0
6,90033.0,44.0,1,1,1,1,0,0.0,0.0,0.0,...,1,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1
9,2109.0,37.0,0,1,1,0,1,0.0,1.0,0.0,...,0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0
10,2109.0,57.0,1,0,0,1,0,1.0,1.0,0.0,...,0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1
13,62791.0,27.0,1,1,0,1,0,0.0,0.0,0.0,...,1,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1
15,930134.0,43.0,1,1,1,0,1,1.0,1.0,0.0,...,1,0.0,1.0,1.0,0.0,1.0,0.0,1.0,11.0,1
16,60616.0,43.0,0,1,1,0,1,0.0,1.0,0.0,...,1,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1


In [62]:
# this will work for the integer columns that do not have a non numeric value
df = df[(df['TVStandWallMount'] == 0) | (df['TVStandWallMount'] == 1)]

df[col] = pd.to_numeric(df[col], errors='coerce')    

In [24]:
# Dell Laptop Cleaning
index_names = df[df['DellLaptop'] == ' '].index
df.drop(index_names, inplace=True)

# Convert object type into numeric
df['DellLaptop'] = pd.to_numeric(df['DellLaptop'], errors='coerce')

# filter out anything that is not 1 or 0
df = df[(df['DellLaptop'] == 0) | (df['DellLaptop'] == 1)]

In [69]:
# clean the objects of anything that isnt numeric
# convert that to numeric
# then filter out anything that isnt binary

In [25]:
# clean space values from object cols
def clean_obj_cols(df, col):
    '''
    Get index of rows where column has ' '
    Drop index
    '''
    index_names = df[df[col] == ' '].index
    df.drop(index_names, inplace=True)
    return df

In [26]:
obj_cols = ['XBOX360', 'DellLaptop', 'SonyTV42LED', 'BoseSoundSystem']

# get object columns and iterate over them to clean all object type columns
for i in obj_cols:
    clean_obj_cols(df, i)

In [27]:
# since obj cols have been cleaned
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')    

for col in df.columns:
    print(col)

Series([], Name: ZipCode, dtype: int64)
Series([], Name: CustomerAge, dtype: int64)
Series([], Name: SamsungTV46LED, dtype: int64)
Series([], Name: SonyTV42LED, dtype: int64)
Series([], Name: XBOX360, dtype: int64)
Series([], Name: DellLaptop, dtype: int64)
Series([], Name: BoseSoundSystem, dtype: int64)
Series([], Name: BoseHeadSet, dtype: int64)
Series([], Name: SonyHeadSet, dtype: int64)
Series([], Name: iPod, dtype: int64)
Series([], Name: iPhone, dtype: int64)
Series([], Name: Panasonic50LED, dtype: int64)
Series([], Name: SonyPS4, dtype: int64)
Series([], Name: WiiU, dtype: int64)
Series([], Name: WDexternalHD, dtype: int64)
Series([], Name: SamsungTV55LED, dtype: int64)
Series([], Name: SonyTV60LED, dtype: int64)
Series([], Name: SandiskMemoryCard, dtype: int64)
Series([], Name: SonySoundSystem, dtype: int64)
Series([], Name: SonyCamera, dtype: int64)
Series([], Name: PanasonicCamera, dtype: int64)
Series([], Name: HPPrinter, dtype: int64)
Series([], Name: SonyDVDplayer, dtype: 

Series([], Name: DellLaptop, dtype: int64)

In [49]:


def return_binary_rows(df, col):
    df = df[(df[col] == 0) | (df[col] == 1)]
    return df


In [50]:
return_binary_rows(df, 'TVStandWallMount')

,ZipCode,CustomerAge,SamsungTV46LED,SonyTV42LED,XBOX360,DellLaptop,BoseSoundSystem,BoseHeadSet,SonyHeadSet,iPod,...,GalaxyTablet,SurfaceTablet,HPLaptop,HDMICable,SpeakerCable,CallOfDutyGame,GrandTheftAutoGame,ASUSLaptop,LenevoLaptop,TVStandWallMount
0,30134.0,35.0,1,1,1,0,0,1.0,1.0,0.0,...,1,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1
1,62791.0,43.0,0,1,0,0,1,0.0,1.0,0.0,...,1,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1
3,60616.0,56.0,0,1,1,1,0,0.0,1.0,1.0,...,0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0
5,2108.0,55.0,1,1,1,1,10,0.0,0.0,0.0,...,1,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0
6,90033.0,44.0,1,1,1,1,0,0.0,0.0,0.0,...,1,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993,90033.0,38.0,1,1,1,0,1,1.0,1.0,0.0,...,1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0
9994,2109.0,27.0,0,1,1,0,1,0.0,1.0,0.0,...,1,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1
9995,33129.0,22.0,1,0,0,1,0,1.0,1.0,0.0,...,1,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1
9996,90024.0,21.0,0,1,1,1,0,0.0,1.0,1.0,...,1,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0


In [99]:
df['TVStandWallMount'].value_counts()


1     6687
0     2689
11      32
10      24
Name: TVStandWallMount, dtype: int64

In [ ]:
#### THIS SHOULD BE CLOSE 
def clean_empty_string(df, col):
    df[df[col] == 1] or df[df[col] != 0]
    return df

for i in df.columns:
    clean_empty_string(df, i)

# this does the same except not in a function
#cleandata4bestdeal = cleandata4bestdeal[cleandata4bestdeal['DellLaptop'] != ' ']

def get_index_names(df, col, val):
    index_names = df[df[col] == val].index
    return index_names

def drop_index_names(df, idx):
    df.drop(idx, inplace=True)
    return df


idx = get_index_names(df, 'DellLaptop', ' ')

drop_index_names(df, idx)

In [ ]:
# This works and gets the index of rows with a certain value
index_names = df[ df['DellLaptop'] == ' '].index

# drop the row indeces for the value you set above
df.drop(index_names, inplace=True)

In [ ]:
#cleandata4bestdeal['XBOX360'].value_counts()
df['DellLaptop'].value_counts()

In [ ]:
cleandata4bestdeal['DellLaptop'].astype(str).astype(int)

In [ ]:
cleandata4bestdeal['DellLaptop'] = cleandata4bestdeal[cleandata4bestdeal['DellLaptop'] != 0] and cleandata4bestdeal[cleandata4bestdeal['DellLaptop'] != 1]

### Lets store the cleaned data into the Database

In [ ]:
engine=create_engine('sqlite:///bestdeal.db')

In [ ]:
cleandata4bestdeal.to_sql('trans4cust', engine)

** Sanity Test: Did it create the table in bestdeal.db?  Check!!**

In [ ]:
insp=inspect(engine)

In [ ]:
 insp.get_table_names()

In [ ]:
pd.read_sql_table('trans4cust', engine).columns

### Now we are ready to query the Database

#### Query example #1: get the transactions for the customers in zipCode 60616

In [ ]:
resultsForBestDealCustTrans=pd.read_sql_query("SELECT * FROM trans4cust WHERE ZipCode='60616'", engine)

In [ ]:
resultsForBestDealCustTrans.head()

#### Query example #2: get the transactions for ALL customers

In [ ]:
resultsForBestDealCustTrans=pd.read_sql_query("SELECT * FROM trans4cust", engine)

In [ ]:
resultsForBestDealCustTrans.head()

#### Query example #3: get the number of customers in every ZipCode sorted by ZipCode

In [ ]:
resultsForBestDealCustTrans=pd.read_sql_query("SELECT ZipCode , COUNT(*) as 'num_customers' FROM trans4cust GROUP BY ZipCode  ORDER BY ZipCode", engine)

In [ ]:
resultsForBestDealCustTrans

#### Query example #4: get the number of customers for  every Age Group in ZipCode 60616 sorted by CustomerAge

In [ ]:
resultsForBestDealCustTrans=pd.read_sql_query(
"SELECT CustomerAge , COUNT(*) as 'num_customers' FROM trans4cust WHERE ZipCode=60616 GROUP BY CustomerAge  ORDER BY CustomerAge", engine)

In [ ]:
resultsForBestDealCustTrans

#### Query example #5: Plot in a stacked-bar figure the number of customers who bought SonyTV60LED and/or BoseSoundSystem in  every zipcode that has more than 400 customers who bought these two products(either bought one of these products or the two products)

In [ ]:
SonyTV60LEDCustTrans=pd.read_sql_query(
"SELECT ZipCode , COUNT(*) as 'num_customers' FROM trans4cust WHERE SonyTV60LED=1  GROUP BY ZipCode HAVING COUNT(*) > 400", engine)

BoseSoundSystemCustTrans=pd.read_sql_query(
"SELECT ZipCode , COUNT(*) as 'num_customers' FROM trans4cust WHERE BoseSoundSystem=1 GROUP BY ZipCode HAVING COUNT(*) > 400", engine)

In [ ]:
SonyTV60LEDCustTrans

In [ ]:
BoseSoundSystemCustTrans

In [ ]:
SonyTV60LEDCustTrans.ZipCode


In [ ]:
import numpy

#   There are zipcodes that Sony got bought but not Bose 
#   but there are also zipcodes that Bose got bought but not Sony
#
#   AND we need to use stacked-bar graph and we have a potentially asymmetrical set  of zipcode values
#   So, we need to do somework to create the symmteric set of zipcode values for Sony and Bose


sonyZipCodeTuples=tuple(SonyTV60LEDCustTrans.ZipCode.astype(numpy.int))
sony_num_customersTuples=tuple(SonyTV60LEDCustTrans.num_customers.astype(numpy.int))

boseZipCodeTuples=tuple(BoseSoundSystemCustTrans.ZipCode.astype(numpy.int))
bose_num_customersTuples=tuple(BoseSoundSystemCustTrans.num_customers.astype(numpy.int))




sony_dict = dict(zip(sonyZipCodeTuples, sony_num_customersTuples))
bose_dict = dict(zip(boseZipCodeTuples, bose_num_customersTuples))

for key in bose_dict.keys():
    if ((key in sony_dict.keys()) == False): sony_dict[key]=0

        
for key in sony_dict.keys():
    if ((key in bose_dict.keys()) == False): bose_dict[key]=0


        
bose_zip= sorted(bose_dict.keys())

sony_zip= sorted(sony_dict.keys())

bose_zip_tuple=tuple(bose_zip)

sony_zip_tuple=tuple(sony_zip)

bose_customer_list=[]

for bose in bose_zip_tuple:
    bose_customer_list.append(bose_dict[bose])

sony_customer_list=[]

for sony in sony_zip_tuple:
    sony_customer_list.append(sony_dict[sony])

bose_customer_tuple=tuple(bose_customer_list)
sony_customer_tuple=tuple(sony_customer_list)


In [ ]:
# See docs for bar_stack at the URL
# http://matplotlib.org/examples/pylab_examples/bar_stacked.html

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline 


ind = np.arange(len(sony_customer_tuple))    


# the width of the bars: can also be len(x) sequence
width = .5


p1 = plt.bar(ind, sony_customer_tuple, width,  color='r')
p2 = plt.bar(ind, bose_customer_tuple, width, color='y', bottom=sony_customer_tuple)


plt.ylabel('Number of Customers')
plt.xlabel('Zip Code')

plt.title('Number of Customers by ZipCode and 2 Products')

plt.xticks(ind + width, sony_zip_tuple, horizontalalignment='right')

plt.yticks(np.arange(0, 2000, 100))
plt.legend((p1[0], p2[0]), ('Sony', 'Bose'))

plt.show()

# Requirements :
1. (Use SQL/SQlite): get the number of customers who bought DellLaptop and HPPrinter for every Age group sorted by CustomerAge
2. (Use SQL/SQlite): Get the list of ZipCodes where no customer bought XBOX360 (this query means NOT even a single csutomer in that zip code bought XBOX360)
3. (Use SQL/SQlite/Matplotlib): Plot in a stacked-bar figure the number of customers who bought HPLaptop and/or HPPrinter but did NOT buy  WDexternalHD for  every CustomerAge group that has more than 100 customers who bought these two products(either bought one of these products or the two products but didn't buy WDexternalHD)


In [ ]:
# Write your python code that meets the above requirements in this cell
